
# **Install libraries**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets tqdm pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
!pip install sentencepiece

In [ ]:
!pip install transformers

In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm

In [ ]:
# Check we have a GPU and check the memory size of the GPU
!nvidia-smi

Thu May  2 04:01:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# **Import packages**

In [ ]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# **Set a seed**

In [ ]:
import random
import numpy as np
import torch
import datasets

In [ ]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

set_seed(42)



```
# This is formatted as code
```

# ***C4-200M dataset***

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/NLP/c4_200m_550k.csv')
df.shape

(550000, 2)

In [ ]:
df.head()

,input,output
0,The steps below describe how to remove data for one or more specifies areas and how to put on the data from a snapshot to the index,The steps below describe how to remove data for one ore more specific areas and how to put back the data from a snapshot to the index.
1,When I wake up it\'s usually comes out dreamsI\'m thinking so my thoughts are very weird.,When I wake up it\'s usually dreams I\'m thinking about so my thoughts are very weird.
2,One of the cardinal factors to be considered trying to decide on which kind of shipping to customer settle is the! market difference.,One of the cardinal factors to consider when trying to decide on which kind of shipping to settle for is the market difference.
3,Answers » Regions » Is in Nagorno-Karabakt region that part in Armenia?,Answers » Regions » Is Nagorno-Karabakh region part of Armenia?
4,Flaneuring in fun at maple creek SK!,Flaneuring Fun in Maple Creek SK!


In [ ]:
from transformers import (
    T5ForConditionalGeneration, T5Tokenizer,
    Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
  )

from torch.utils.data import Dataset, DataLoader

In [ ]:
model_name = 't5-base'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def calc_token_len(example):
    return len(tokenizer(example).input_ids)

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.10, shuffle=True)
train_df.shape, test_df.shape

((495000, 2), (55000, 2))

In [ ]:
test_df['input_token_len'] = test_df['input'].apply(calc_token_len)

In [ ]:
test_df.head()

,input,output,input_token_len
486496,You are My Fantasy. And My Reality. So good. ...Sated.,"You are My Fantasy. And, My Reality. So good. ...Sated.",18
58208,I have always found it be good with.,I have always found it to be good.,10
265247,"The average rock you would pick up has an SG of about 2.75 because all the earth’s bulk crust is made up of quartz, calcite & feldspar.","The average rock you would pick up has an SG of about 2.75. Because most of the earth’s crust is made up of quartz, calcite & feldspar.",40
499297,"Bronzes, Mirrors and paintings, fine some art of Sydney.","Bronzes, Mirrors and Paintings, some of the finest in Sydney.",15
124684,Is this how america became under Donald Trump?,Is this what America has become under Donald Trump?,12


In [ ]:
test_df['input_token_len'].describe()

count    55000.000000
mean        33.536127
std         25.961464
min          2.000000
25%         17.000000
50%         27.000000
75%         42.000000
max        918.000000
Name: input_token_len, dtype: float64

### We will use a token length of 64 since it will cover the vast majority of examples

In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
test_dataset

Dataset({
    features: ['input', 'output', 'input_token_len', '__index_level_0__'],
    num_rows: 55000
})

### Load the Dataset

In [ ]:
from torch.utils.data import Dataset, DataLoader
class GrammarDataset(Dataset):
    def __init__(self, dataset, tokenizer,print_text=False):
        self.dataset = dataset
        self.pad_to_max_length = False
        self.tokenizer = tokenizer
        self.print_text = print_text
        self.max_len = 64

    def __len__(self):
        return len(self.dataset)


    def tokenize_data(self, example):
        input_, target_ = example['input'], example['output']

        # tokenize inputs
        tokenized_inputs = tokenizer(input_, pad_to_max_length=self.pad_to_max_length,
                                            max_length=self.max_len,
                                            return_attention_mask=True)

        tokenized_targets = tokenizer(target_, pad_to_max_length=self.pad_to_max_length,
                                            max_length=self.max_len,
                                            return_attention_mask=True)

        inputs={"input_ids": tokenized_inputs['input_ids'],
            "attention_mask": tokenized_inputs['attention_mask'],
            "labels": tokenized_targets['input_ids']
        }

        return inputs


    def __getitem__(self, index):
        inputs = self.tokenize_data(self.dataset[index])

        if self.print_text:
            for k in inputs.keys():
                print(k, len(inputs[k]))

        return inputs

In [ ]:
dataset = GrammarDataset(test_dataset, tokenizer, True)
print(dataset[121])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


input_ids 20
attention_mask 20
labels 24
{'input_ids': [71, 973, 24, 14079, 24067, 38, 96, 77, 221, 3728, 121, 19, 59, 2930, 7509, 640, 569, 2287, 5, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [71, 973, 24, 14079, 3, 89, 12578, 887, 21, 96, 77, 221, 3728, 121, 3270, 19, 59, 2930, 7509, 640, 569, 2287, 5, 1]}


### Define Evaluator

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=8d02445178a415dbf5693459053fa41c10e4cb4e3caf2622f09dd5d4c2e94c6a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
from datasets import load_metric
rouge_metric = load_metric("rouge")

<ipython-input-29-76fa4d2ef6b0>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


### Train Model

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest', return_tensors='pt')

In [ ]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
# defining training related arguments
batch_size = 16
args = Seq2SeqTrainingArguments(output_dir="/content/drive/shared-with-me/c4_200m/weights",
                        evaluation_strategy="steps",
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        learning_rate=2e-5,
                        num_train_epochs=1,
                        weight_decay=0.01,
                        save_total_limit=2,
                        predict_with_generate=True,
                        fp16 = True,
                        gradient_accumulation_steps = 6,
                        eval_steps = 500,
                        save_steps = 500,
                        load_best_model_at_end=True,
                        logging_dir="/logs",
                        report_to="wandb")

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
import nltk
nltk.download('punkt')
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# defining trainer using 🤗
trainer = Seq2SeqTrainer(model=model,
                args=args,
                train_dataset= GrammarDataset(train_dataset, tokenizer),
                eval_dataset=GrammarDataset(test_dataset, tokenizer),
                tokenizer=tokenizer,
                data_collator=data_collator,
                compute_metrics=compute_metrics)

Using amp half precision backend


In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 495000
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 96
  Gradient Accumulation steps = 6
  Total optimization steps = 5156
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
500,0.772300,0.626797,71.379300,60.962000,70.628300,70.652400,17.308300
1000,0.668700,0.598719,71.688800,61.516800,70.955200,70.984700,17.281400
1500,0.655300,0.585948,71.858300,61.810200,71.133800,71.162000,17.267000
2000,0.642800,0.576314,71.923500,61.937400,71.198000,71.226400,17.263400


***** Running Evaluation *****
  Num examples = 55000
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/c4_200m/weights/checkpoint-500
Configuration saved in /content/drive/MyDrive/c4_200m/weights/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/c4_200m/weights/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/c4_200m/weights/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/c4_200m/weights/checkpoint-500/special_tokens_map.json
Deleting older checkpoint [/content/drive/MyDrive/c4_200m/weights/checkpoint-4500] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 55000
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/c4_200m/weights/checkpoint-1000
Configuration saved in /content/drive/MyDrive/c4_200m/weights/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/c4_200m/weights/checkpoint-1000/pytorch_model.b

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
500,0.772300,0.626797,71.379300,60.962000,70.628300,70.652400,17.308300
1000,0.668700,0.598719,71.688800,61.516800,70.955200,70.984700,17.281400
1500,0.655300,0.585948,71.858300,61.810200,71.133800,71.162000,17.267000
2000,0.642800,0.576314,71.923500,61.937400,71.198000,71.226400,17.263400
2500,0.632500,0.571061,71.994700,62.064300,71.280800,71.308900,17.254900
3000,0.624400,0.568164,72.064800,62.185500,71.355200,71.382600,17.250500
3500,0.619500,0.563820,72.080500,62.221000,71.374300,71.401900,17.249000
4000,0.625300,0.562196,72.104600,62.255800,71.399100,71.425700,17.246500
4500,0.617400,0.561505,72.114500,62.274700,71.409300,71.436700,17.245400
5000,0.612200,0.561453,72.110600,62.275600,71.406600,71.434300,17.245100


Saving model checkpoint to /content/drive/MyDrive/c4_200m/weights/checkpoint-2500
Configuration saved in /content/drive/MyDrive/c4_200m/weights/checkpoint-2500/config.json
Model weights saved in /content/drive/MyDrive/c4_200m/weights/checkpoint-2500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/c4_200m/weights/checkpoint-2500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/c4_200m/weights/checkpoint-2500/special_tokens_map.json
Deleting older checkpoint [/content/drive/MyDrive/c4_200m/weights/checkpoint-1500] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 55000
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/c4_200m/weights/checkpoint-3000
Configuration saved in /content/drive/MyDrive/c4_200m/weights/checkpoint-3000/config.json
Model weights saved in /content/drive/MyDrive/c4_200m/weights/checkpoint-3000/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/c4_200m/we

TrainOutput(global_step=5156, training_loss=0.6461646921233303, metrics={'train_runtime': 41087.7768, 'train_samples_per_second': 12.047, 'train_steps_per_second': 0.125, 'total_flos': 3.645646300422144e+16, 'train_loss': 0.6461646921233303, 'epoch': 1.0})

In [ ]:
trainer.save_model('t5_gec_model')

Saving model checkpoint to t5_gec_model
Configuration saved in t5_gec_model/config.json
Model weights saved in t5_gec_model/pytorch_model.bin
tokenizer config file saved in t5_gec_model/tokenizer_config.json
Special tokens file saved in t5_gec_model/special_tokens_map.json


In [ ]:
!zip -r 't5_gec_model.zip' 't5_gec_model'

  adding: t5_gec_model/ (stored 0%)
  adding: t5_gec_model/special_tokens_map.json (deflated 83%)
  adding: t5_gec_model/training_args.bin (deflated 48%)
  adding: t5_gec_model/config.json (deflated 62%)
  adding: t5_gec_model/tokenizer_config.json (deflated 80%)
  adding: t5_gec_model/pytorch_model.bin (deflated 8%)
  adding: t5_gec_model/spiece.model (deflated 48%)


In [ ]:
!mv t5_gec_model.zip /content/drive/MyDrive/c4_200m

I have uploaded this model to HuggingFace Model Zoo and we can run inference using it

## Testing

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
model_name = 'deep-learning-analytics/GrammarCorrector'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def correct_grammar(input_text,num_return_sequences):
  batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [ ]:
text = 'He are moving here.'
print(correct_grammar(text, num_return_sequences=2))

['He is moving here.', "He's moving here."]


In [ ]:
text = 'Cat drinked milk'
print(correct_grammar(text, num_return_sequences=1))

['Cat drank milk. Cat drank milk.']
